In [261]:
from bs4 import BeautifulSoup
import requests, csv, string
from csv import writer

filename = "operas_masterlist.csv"
csv = open(filename, "w") 
# columnTitleRow = 'composer,composer_dob,opera'
# csv.write(columnTitleRow)

operas = []
for letter in list(string.ascii_uppercase):
    url = "http://opera.stanford.edu/composers/" + letter + ".html"
    r  = requests.get(url)
    soup = BeautifulSoup(r.text, "html5lib")
    composers = soup.find_all('li')    
    for composer in composers:
        if composer.find('b'):
            comp = composer.find('b').text
            try:
                dob = list(composer)[1].strip()
            except:
                dob = ''
            try:
                works = composer.find('ul').find_all('li')
                for w in works:
                    work = []
                    work.append(comp)
                    work.append(dob)                    
                    work.append(w.text.strip())   
                    operas.append(work)                
            except:
                None
                
with open("operas_masterlist.csv", 'w') as outfile:
    mywriter = writer(outfile)
    mywriter.writerow(['composer', 'composer_dob', 'opera'])
    for opera in operas:
        mywriter.writerow(opera)                

In [269]:
# with open('operas_masterlist.csv', 'r') as r, open('operas_masterlist1.csv', 'w') as w:    
#     for num, line in enumerate(r):
#         if num > 0:            
#             newline = line[:-2] + "\n" if "\n" in line else line[:-1]
#         else:
#             newline = line               
#         w.write(newline)

In [304]:

# operas.head()
# operas.columns

In [382]:
import re 
import pandas as pd

filename = "operas_masterlist.csv"
operas = pd.read_csv(filename, lineterminator='\n', encoding='utf-8')

def get_birth_year(text):
    birth = ""
    str = text.split('***')
    if len(str) > 0:      
        year = re.findall(r"(\d{4})", str[0])
        if len(year) > 0:
            birth = year[0]
    return birth

def get_death_year(text):
    birth = ""
    str = text.split('***')
    if len(str) > 0:      
        year = re.findall(r"(\d{4})", str[-1])
        if len(year) > 0:
            birth = year[0]
    return birth

def get_birth_place(text):
    birth = ""
    str = text.split('***')
    if len(str) > 0:      
        place = re.findall(r"[a-zA-Z\x7f-\xff]+", str[0])
        if len(place) > 0:
            birth = place[0]
    return birth

def get_death_place(text):
    birth = ""
    str = text.split('***')
    if len(str) > 0:      
        place = re.findall(r"[a-zA-Z\x7f-\xff]+", str[-1])
        if len(place) > 0:
            birth = place[0]
    return birth

def get_work_date(text):
    try:
        year = re.findall(r"(\d{4})", text)[-1]
    except:
        year = 1000
    return year


def get_work_name(text):
    name = text
    str = text.split('(')
    if len(str) > 0:      
        try: 
            name = str[0]
        except:
            None
    return name

operas['opera'] = operas.apply(lambda row: row['opera\r'], axis=1)
operas = operas.drop(['opera\r'], axis=1)
operas = operas.replace('\r','', regex=True)
operas['composer_dob'] = operas['composer_dob'].astype('str')
operas['dob'] = operas.apply(lambda row: row['composer_dob'], axis=1)
operas['dob'] = operas.apply(lambda row: row['dob'].replace("(", ''), axis=1)
operas['dob'] = operas.apply(lambda row: row['dob'].replace(")", ''), axis=1)
operas['dob'] = operas.apply(lambda row: row['dob'].lower(), axis=1)
operas['dob'] = operas.apply(lambda row: row['dob'].replace("c.", ''), axis=1)
operas['dob'] = operas.apply(lambda row: row['dob'].replace("'s", ''), axis=1)
operas['dob'] = operas.apply(lambda row: row['dob'].replace("st ", 'st_'), axis=1)
operas['dob'] = operas.apply(lambda row: row['dob'].replace("s. ", 'st_'), axis=1)
operas['dob'] = operas.apply(lambda row: row['dob'].replace("l'", 'l'), axis=1)
operas['dob'] = operas.apply(lambda row: row['dob'].replace("la ", 'la_'), axis=1)



operas['dob'] = operas.apply(lambda row: row['dob'].replace(' - ', '***'), axis=1)
operas['year_birth'] = operas.apply(lambda row: get_birth_year(row['dob']), axis=1)
operas['year_death'] = operas.apply(lambda row: get_death_year(row['dob']), axis=1)
operas['place_birth'] = operas.apply(lambda row: get_birth_place(row['dob']), axis=1)
operas['place_death'] = operas.apply(lambda row: get_death_place(row['dob']), axis=1)
operas['work_date'] = operas.apply(lambda row: get_work_date(row['opera']), axis=1)
operas['work_name'] = operas.apply(lambda row: get_work_name(row['opera']), axis=1)
operas.to_csv('operas_masterlist_processed.csv')
operas.head()

,composer,composer_dob,opera,dob,year_birth,year_death,place_birth,place_death,work_date,work_name
0,"Aav, Evald",(7.3.1900 Reval - 21.3.1939 Tallinn),Vikerlased (8.9.1928 Tallinn),7.3.1900 reval***21.3.1939 tallinn,1900,1939,reval,tallinn,1928,Vikerlased
1,"Abbà Cornaglia, Pietro",(20.3.1851 Alessandria - 2.5.1894 Alessandria),Isabella Spinola (1877 Milano),20.3.1851 alessandria***2.5.1894 alessandria,1851,1894,alessandria,alessandria,1877,Isabella Spinola
2,"Abbà Cornaglia, Pietro",(20.3.1851 Alessandria - 2.5.1894 Alessandria),Maria di Warden (1884 Milano),20.3.1851 alessandria***2.5.1894 alessandria,1851,1894,alessandria,alessandria,1884,Maria di Warden
3,"Abbà Cornaglia, Pietro",(20.3.1851 Alessandria - 2.5.1894 Alessandria),Una partita a scacchi (1892 Pavia),20.3.1851 alessandria***2.5.1894 alessandria,1851,1894,alessandria,alessandria,1892,Una partita a scacchi
4,"Abbadia, Natale",(11.3.1792 Genova - 25.12.1861 Milano),Giannina di Pontieu (1812),11.3.1792 genova***25.12.1861 milano,1792,1861,genova,milano,1812,Giannina di Pontieu


In [387]:
operas[operas["composer"] == "Delmas, Marc-Jean-Baptiste"]['dob']

6189    28.3.1885 st_quentin***30.11.1931 paris
6190    28.3.1885 st_quentin***30.11.1931 paris
6191    28.3.1885 st_quentin***30.11.1931 paris
6192    28.3.1885 st_quentin***30.11.1931 paris
6193    28.3.1885 st_quentin***30.11.1931 paris
6194    28.3.1885 st_quentin***30.11.1931 paris
6195    28.3.1885 st_quentin***30.11.1931 paris
6196    28.3.1885 st_quentin***30.11.1931 paris
6197    28.3.1885 st_quentin***30.11.1931 paris
6198    28.3.1885 st_quentin***30.11.1931 paris
6199    28.3.1885 st_quentin***30.11.1931 paris
Name: dob, dtype: object

In [347]:
# operas[['work_name', 'opera']]
# 

In [282]:
# re.search(r"(\d{4})", '7.3.1900 Reval')(0)
# re.findall(r"(\d{4})", '7.3.1900 Reval')

In [306]:
# operas = operas.drop(['opera\r'], axis=1)
# operas.columns